In [ ]:
from dataclasses import dataclass
from datetime import datetime
from enum import Enum
import uuid

class ServiceType(Enum):
    FLIGHT = "flight"
    HOTEL = "hotel"
    CAR_RENTAL = "car_rental"

class MCPType(Enum):
    WEB = "web"
    DATABASE = "database"
    AUTH = "authentication"

class AgentStatus(Enum):
    ONLINE = "online"
    OFFLINE = "offline"
    MAINTENANCE = "maintenance"

@dataclass
class AgentMessage:
    message_id: str
    sender_id: str
    recipient_id: str
    timestamp: datetime
    payload: dict
    message_type: ServiceType
    requires_response: bool = True

class MCPServer:
    def __init__(self, server_type: MCPType):
        self.server_type = server_type
        self.airline_agent = None
        self.mock_prices = {
            ServiceType.FLIGHT: 300,
            ServiceType.HOTEL: 150,
        }
        self.status = AgentStatus.ONLINE

    def handle_request(self, message: AgentMessage) -> dict:
        if self.status != AgentStatus.ONLINE:
            return self._error_response("Server unavailable")

        if self.server_type == MCPType.WEB:
            return self._handle_web_request(message)
        elif self.server_type == MCPType.DATABASE:
            return self._handle_db_request(message)
        return self._error_response("Invalid server type")

    def _handle_web_request(self, message: AgentMessage) -> dict:
        # Forward to appropriate handler
        if message.message_type == ServiceType.FLIGHT:
            return self.airline_agent.db_mcp.handle_request(message)
        return self._error_response("Unsupported service type")

    def _handle_db_request(self, message: AgentMessage) -> dict:
        service = message.message_type
        return {
            "status": "success",
            "price": self.mock_prices.get(service, 0),
            "currency": "USD",
            "valid_until": datetime.now().isoformat()
        }

    def _error_response(self, error_msg: str) -> dict:
        return {"status": "error", "message": error_msg}

class AgentCard:
    def __init__(self, agent_id: str, name: str,
                 web_mcp: MCPServer, db_mcp: MCPServer,
                 services: list[ServiceType]):
        self.agent_id = agent_id
        self.name = name
        self.web_mcp = web_mcp
        self.db_mcp = db_mcp
        self.services = services
        self.last_contacted = None
        self.status = AgentStatus.ONLINE
        self.auth_token = str(uuid.uuid4())
        self.service_level = 1  # 1-3 priority level

    def update_status(self, status: AgentStatus):
        self.status = status
        self.web_mcp.status = status
        self.db_mcp.status = status

    def generate_message(self, recipient: 'AgentCard',
                        service_type: ServiceType, payload: dict) -> AgentMessage:
        return AgentMessage(
            message_id=str(uuid.uuid4()),
            sender_id=self.agent_id,
            recipient_id=recipient.agent_id,
            timestamp=datetime.now(),
            payload=payload,
            message_type=service_type
        )

    def validate_response(self, response: dict) -> bool:
        return response.get('status') == "success"

class AirlineAgent:
    def __init__(self, agent_card: AgentCard):
        self.agent_card = agent_card
        # Link MCP servers back to this agent
        self.agent_card.web_mcp.airline_agent = self
        self.agent_card.db_mcp.airline_agent = self

class BookingAgent:
    def __init__(self, agent_id: str, name: str):
        self.agent_card = AgentCard(
            agent_id=agent_id,
            name=name,
            web_mcp=MCPServer(MCPType.WEB),
            db_mcp=MCPServer(MCPType.DATABASE),
            services=[ServiceType.FLIGHT, ServiceType.HOTEL]
        )
        self.airline_agents = []

    def add_airline_agent(self, airline_agent: AirlineAgent):
        self.airline_agents.append(airline_agent)

    def find_best_option(self, service_type: ServiceType):
        options = []

        for airline_agent in self.airline_agents:
            agent_card = airline_agent.agent_card
            if service_type not in agent_card.services:
                continue

            message = self.agent_card.generate_message(
                recipient=agent_card,
                service_type=service_type,
                payload={"query": "price_check"}
            )

            try:
                response = agent_card.web_mcp.handle_request(message)
                if agent_card.validate_response(response):
                    options.append({
                        "agent": agent_card.name,
                        "price": response.get('price'),
                        "currency": response.get('currency')
                    })
            except Exception as e:
                print(f"Error contacting {agent_card.name}: {str(e)}")

        return sorted(options, key=lambda x: x['price'])

# Example Usage
if __name__ == "__main__":
    # Setup Emirates
    emirates_web = MCPServer(MCPType.WEB)
    emirates_db = MCPServer(MCPType.DATABASE)
    emirates_db.mock_prices[ServiceType.FLIGHT] = 300
    emirates_card = AgentCard(
        agent_id="C",
        name="Emirates",
        web_mcp=emirates_web,
        db_mcp=emirates_db,
        services=[ServiceType.FLIGHT, ServiceType.HOTEL]
    )
    emirates_agent = AirlineAgent(emirates_card)

    # Setup AirIndia
    airindia_web = MCPServer(MCPType.WEB)
    airindia_db = MCPServer(MCPType.DATABASE)
    airindia_db.mock_prices[ServiceType.FLIGHT] = 250
    airindia_card = AgentCard(
        agent_id="E",
        name="AirIndia",
        web_mcp=airindia_web,
        db_mcp=airindia_db,
        services=[ServiceType.FLIGHT]
    )
    airindia_agent = AirlineAgent(airindia_card)

    # Configure Booking Agent A
    agent_a = BookingAgent("A", "TravelMaster")
    agent_a.add_airline_agent(emirates_agent)
    agent_a.add_airline_agent(airindia_agent)

    # Find best flight option
    flights = agent_a.find_best_option(ServiceType.FLIGHT)
    print("Best flight options:")
    for idx, option in enumerate(flights, 1):
        print(f"{idx}. {option['agent']}: {option['price']} {option['currency']}")

    # Example hotel booking (not supported by AirIndia)
    hotels = agent_a.find_best_option(ServiceType.HOTEL)
    print("\nHotel options:")
    print(hotels)